In [ ]:
from keypoint_dataset import KeypointDataset
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm import tqdm
import torch
import numpy as np

In [ ]:
datasetPath = "/home/giorgio6846/Code/Sign-AI/data/dataset_clean.hdf5"
kd = KeypointDataset(datasetPath, return_label=True, max_length=20000000000)



In [ ]:
import h5py
import matplotlib.pyplot as plt

with h5py.File(datasetPath, "r") as f:
    first_clip = list(f.keys())[0]
    sample = list(f[first_clip]["keypoints"].values())[0][:]
    print("Shape:", sample.shape)


In [ ]:
import matplotlib.pyplot as plt

def plot_keypoints(frame, title="Frame Keypoints"):
    frame = torch.as_tensor(frame, dtype=torch.float32)
    x = frame[:, 0]
    y = -frame[:, 1]  # invertir para que se vea con orientación humana

    plt.figure(figsize=(6, 6))
    plt.scatter(x, y, c='g', label='Mano izq')

    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

plot_keypoints(sample[0], "Sample Keypoints")

In [ ]:
import pandas as pd

# to df
df = pd.DataFrame(sample[1], columns=["x", "y"])
df.shape



In [ ]:
df[
df.duplicated()]



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import torch

def animate_keypoints(frame, interval=50):
    """
    Muestra los keypoints uno por uno con su índice.
    """
    frame = torch.as_tensor(frame, dtype=torch.float32)
    x = frame[:, 0].numpy()
    y = -frame[:, 1].numpy()  # invertimos eje Y para orientación humana

    fig, ax = plt.subplots(figsize=(6, 6))
    scat = ax.scatter([], [], s=20)
    ax.set_xlim(x.min() - 50, x.max() + 50)
    ax.set_ylim(y.min() - 50, y.max() + 50)
    ax.set_title("Keypoints por índice")
    ax.grid(True)

    coords = []
    texts = []

    def update(i):
        coords.append((x[i], y[i]))
        scat.set_offsets(coords)

        # Añadir el índice como texto sobre el keypoint actual
        text = ax.text(x[i], y[i], str(i), fontsize=8, color='red')
        texts.append(text)

        return scat, *texts

    ani = animation.FuncAnimation(fig, update, frames=len(x), interval=interval, repeat=False)
    plt.close(fig)
    return HTML(ani.to_jshtml())

animate_keypoints(sample[0], interval=100)


Hypo keypoints:

0: ref. cara
1: ref cuello
2,3,4 ref hombro der, ref codo der, ref mano

5,6,7: same izq

8 (?) DER
9 (?) IZQ
10 (9)


14-17 cejas


pose = person[0:25,:]

face = person[25:95,:]

hand_l = person[95:116,:]

hand_r = person[0,133,:]



center = (person[1] + person[4]) / 2
scale = scale if scale != 0 else 1.0
normalized = (person - center) / scale



Prepro hh

In [ ]:
len(kd)

---

In [ ]:
minValue = float('inf')
maxValue = float('-inf')
lengths = []

for idx in tqdm(range(len(kd)), desc="Analizando longitudes"):
    keypoints_frames = kd[idx][0].shape[0]  # cantidad de frames
    lengths.append(keypoints_frames)
    minValue = min(keypoints_frames, minValue)
    maxValue = max(keypoints_frames, maxValue)

print(f"Min length: {minValue} frames")
print(f"Max length: {maxValue} frames")
print(f"Promedio: {sum(lengths)/len(lengths):.2f} frames")


In [ ]:

plt.hist(lengths, bins=30)
plt.title("Distribución de longitudes de secuencia")
plt.xlabel("Frames por muestra")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
# Percentil 95 para decidir max_seq_len sin truncar tanto
p95 = int(np.percentile(lengths, 95))
print(f"Sugerencia: usar max_seq_len = {p95} cubre el 95% de los clips")

# Visualización
plt.hist(lengths, bins=30, color='skyblue', edgecolor='black')
plt.axvline(p95, color='red', linestyle='--', label=f'95% ≤ {p95}')
plt.title("Distribución de longitudes de secuencia")
plt.xlabel("Frames por muestra")
plt.ylabel("Frecuencia")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

---

In [ ]:
keypoints = kd[4000][0].clone()
keypoints.shape



In [ ]:
def filter_keypoints_preserve_indices(keypoints, keep_n=130, preserve_idx=None):
    """
    Filtra keypoints por varianza temporal, preservando algunos índices.

    Args:
        keypoints: Tensor (T, N, 2)
        keep_n: número total de keypoints que quieres conservar
        preserve_idx: lista de índices que siempre deben conservarse (ej. manos, rostro)

    Returns:
        filtered: Tensor (T, N, 2) con puntos fuera de la máscara puestos en 0
        final_mask: Tensor (N,) booleano indicando puntos válidos
    """
    T, N, _ = keypoints.shape
    if preserve_idx is None:
        preserve_idx = []

    # Calcula varianza por keypoint (temporalmente)
    var = keypoints.var(dim=0).mean(dim=1)  # (N,)

    # Los que ya están preservados
    preserve_mask = torch.zeros(N, dtype=torch.bool)
    preserve_mask[preserve_idx] = True

    # De los que NO están preservados, ordena por varianza
    num_to_select = max(keep_n - len(preserve_idx), 0)
    rest_idx = [i for i in range(N) if i not in preserve_idx]
    rest_var = var[rest_idx]
    _, top_rest_idx = torch.topk(-rest_var, k=num_to_select)  # -var para menor varianza
    selected_rest = [rest_idx[i.item()] for i in top_rest_idx]

    # Unión de preservados + seleccionados
    final_mask = torch.zeros(N, dtype=torch.bool)
    final_mask[preserve_idx] = True
    final_mask[selected_rest] = True

    # Aplicar la máscara
    filtered = keypoints.clone()
    for i in range(N):
        if not final_mask[i]:
            filtered[:, i] = 0

    return filtered, final_mask


In [ ]:
f_keypoint = filter_keypoints_preserve_indices(keypoints,)

f_keypoint = keypoints

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))  # ax será ahora un array con 2 elementos
ax[0].set_title('Keypoints Originales')
ax[1].set_title('Keypoints Filtrados')

sc1 = ax[0].scatter(keypoints[0, :, 0], keypoints[0, :, 1], s=10, c='blue', alpha=0.5)
sc2 = ax[1].scatter(f_keypoint[0, :, 0], f_keypoint[0, :, 1], s=10, c='red', alpha=0.5)
for a in ax:
    #ax.set_xlim(0, 1)
    #ax.set_ylim(0, 1)
    a.grid(True)
    a.set_aspect('equal')
    a.invert_yaxis()  # Invertir eje Y para que coincida con la convención de coordenadas

In [ ]:
def update(frame):
    points = keypoints[frame]
    filtered_points = f_keypoint[frame]
    sc1.set_offsets(points)
    sc2.set_offsets(filtered_points)
    ax[0].set_title(f'Frame {frame} - Original')
    ax[1].set_title(f'Frame {frame} - Filtrado')
    return sc1, sc2

In [ ]:
anim = FuncAnimation(fig, update, frames=keypoints.shape[0], interval=50, blit=True)
HTML(anim.to_jshtml())

In [ ]:
mean_kp = keypoints.mean(axis=0)  # (N, 2)
plt.scatter(mean_kp[:, 0], mean_kp[:, 1])
